# Assignment 2

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

An NOAA dataset has been stored in the file `data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv`. The data for this assignment comes from a subset of The National Centers for Environmental Information (NCEI) [Daily Global Historical Climatology Network](https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt) (GHCN-Daily). The GHCN-Daily is comprised of daily climate records from thousands of land surface stations across the globe.

Each row in the assignment datafile corresponds to a single observation.

The following variables are provided to you:

* **id** : station identification code
* **date** : date in YYYY-MM-DD format (e.g. 2012-01-24 = January 24, 2012)
* **element** : indicator of element type
    * TMAX : Maximum temperature (tenths of degrees C)
    * TMIN : Minimum temperature (tenths of degrees C)
* **value** : data value for element (tenths of degrees C)

For this assignment, you must:

1. Read the documentation and familiarize yourself with the dataset, then write some python code which returns a line graph of the record high and record low temperatures by day of the year over the period 2005-2014. The area between the record high and record low temperatures for each day should be shaded.
2. Overlay a scatter of the 2015 data for any points (highs and lows) for which the ten year record (2005-2014) record high or record low was broken in 2015.
3. Watch out for leap days (i.e. February 29th), it is reasonable to remove these points from the dataset for the purpose of this visualization.
4. Make the visual nice! Leverage principles from the first module in this course when developing your solution. Consider issues such as legends, labels, and chart junk.

The data you have been given is near **Ann Arbor, Michigan, United States**, and the stations the data comes from are shown on the map below.

In [ ]:
!find ~ | grep -i C2A2_data

In [ ]:
!head ../../_data/ghcnd-stations.txt

In [ ]:
import matplotlib.pyplot as plt
import mplleaflet
import pandas as pd

def leaflet_plot_stations(binsize, hashid):

    df = pd.read_csv('data/C2A2_data/BinSize_d{}.csv'.format(binsize))

    station_locations_by_hash = df[df['hash'] == hashid]

    lons = station_locations_by_hash['LONGITUDE'].tolist()
    lats = station_locations_by_hash['LATITUDE'].tolist()

    plt.figure(figsize=(8,8))

    plt.scatter(lons, lats, c='r', alpha=0.7, s=200)

    return mplleaflet.display()

leaflet_plot_stations(400,'fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89')

In [ ]:
# Import data
df_GHCN = pd.read_csv('data/C2A2_data/BinnedCsvs_d400/fb441e62df2d58994928907a91895ec62c2c42e6cd075c2700843b89.csv')

In [ ]:
# Convert long DF to wide DF; TMIN and TMAX to columns
df_GHCN_max = df_GHCN[df_GHCN['Element'] == 'TMAX']
df_GHCN_min = df_GHCN[df_GHCN['Element'] == 'TMIN']
df_GHCN_max.head(50)
df_GHCN = pd.merge(df_GHCN_min, df_GHCN_max, how='outer', left_on=['ID','Date'], right_on=['ID','Date'])
df_GHCN.columns = ['ID', 'Date', 'x', 'TMIN', 'y', 'TMAX']
df_GHCN.drop(['x', 'y'], axis=1, inplace=True)
df_GHCN.head(10)

In [ ]:
# Convert dates
# observation_dates = np.arange('2017-01-01', '2017-01-09', dtype='datetime64[D]')
df_GHCN['Date'] = list(map(pd.to_datetime, df_GHCN['Date'])) 
df_GHCN['Day of Year'] = df_GHCN['Date'].dt.dayofyear
df_GHCN['Day'] = df_GHCN['Date'].dt.day
df_GHCN['Month'] = df_GHCN['Date'].dt.month
df_GHCN['Year'] = df_GHCN['Date'].dt.year
df_GHCN.dtypes

# Mask leap day
mask_leap = (df_GHCN['Date'].dt.month == 2) & (df_GHCN['Date'].dt.day == 29)
df_GHCN = df_GHCN[~mask_leap]

# Sanity check 
print(df_GHCN[mask_leap])



df_GHCN.head(10)

In [ ]:
# 
df_GHCN['Day Min'] = df_GHCN.groupby('Day of Year')['TMIN'].transform('min')
df_GHCN['Day Max'] = df_GHCN.groupby('Day of Year')['TMAX'].transform('max')
df_GHCN.head(-1)

In [ ]:
# mpl.rcParams['agg.path.chunksize'] = 100000 # https://stackoverflow.com/questions/37470734/matplotlib-giving-error-overflowerror-in-draw-path-exceeded-cell-block-limit
plt.figure(figsize=(16, 6))
plt.plot(df_GHCN['Day of Year'], df_GHCN.groupby('Day of Year')['TMIN'].transform('min')) #, '-o')#,  df_GHCN['Day'], df_GHCN['Day Max'], '-o')
plt.show()